In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras import layers
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils import pad_sequences
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Masking

In [4]:
df = pd.read_csv('/Users/hannadinizcunha/Downloads/data.csv', index_col='index')

In [5]:
df

,text,isfake
index,,
0,head conservative republican faction us congre...,0
1,transgender people allow first time enlist us ...,0
2,president donald trump call us postal service ...,0
3,west palm beach flathe white house say friday ...,0
4,west palm beach fla president donald trump say...,0
...,...,...
11754,paul craig robertsin last years th century fra...,1
11755,robert fantina counterpunchalthough unite stat...,1
11756,st century wire say wire report earlier week u...,1


In [6]:
X = df.text.values
y = df.isfake.values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [8]:
X_train_split= [text_to_word_sequence(t) for t in X_train]

X_test_split= [text_to_word_sequence(t) for t in X_test]

In [9]:
w2v= Word2Vec(sentences= X_train_split, vector_size=1, window=5)
word= w2v.wv

In [10]:
# Convert the text into a matrix of the words in the embedding space
def embed_sentence(w2v, text):
    embedded_text = []
    for words in text:
        if words in w2v.wv:
            embedded_text.append(w2v.wv[words])

    return np.array(embedded_text)

# Function that converts a list of sentences into a list of matrices
def embedding(w2v, texts):
    embed = []

    for text in texts:
        embedded_text = embed_sentence(w2v, text)
        embed.append(embedded_text)

    return embed

# Embed the training and test sentences
X_train_embed = embedding(w2v, X_train_split)
X_test_embed = embedding(w2v, X_test_split)

In [11]:
# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed,truncating='post', maxlen=500)
X_test_pad = pad_sequences(X_test_embed, truncating='post', maxlen=500)

In [12]:
# Reducing dimension
new_train = np.mean(X_train_pad, axis=2)
new_test = np.mean(X_test_pad, axis=2)

In [13]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0,
)
tree_clf.fit(new_train,y_train)

y_pred = tree_clf.predict(new_test)
accuracy_score(y_test, y_pred)

0.7098214285714286

In [20]:
# RandomForestClassifier & LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier()
logreg = LogisticRegression(max_iter=2000)

ensemble = VotingClassifier(
    estimators = [("rf", forest),("lr", logreg)],
    voting = 'soft', # to use predict_proba of each classifier before voting
    weights = [1,1] # to equally weight forest and logreg in the vote
)
ensemble.fit(new_train,y_train)

y_pred_RFLR = ensemble.predict(new_test)
accuracy_score(y_test, y_pred_RFLR)

0.7553146258503401

In [18]:
# RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(new_train,y_train)

y_pred_RFC = clf.predict(new_test)
accuracy_score(y_test, y_pred_RFC)

0.7848639455782312

In [19]:
RandomForestClassifier(
    n_estimators=100,
    *,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='sqrt',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None,
    ccp_alpha=0.0,
    max_samples=None,
)

clf.fit(new_train,y_train)

y_pred_RFC = clf.predict(new_test)
accuracy_score(y_test, y_pred_RFC)

SyntaxError: iterable argument unpacking follows keyword argument unpacking (2219506735.py, line 2)